In [ ]:
!python3 -m pip install google-adk==1.2.1

In [ ]:
!pip install crewai

In [ ]:
# main.py
from crewai import Crew
from agents  import schema_extractor_agent, object_metadata_agent, bq_mapper_agent
from tasks import AgentTasks
from langchain_core.utils.function_calling import convert_to_openai_tool
from tools.schema_listing_tool import list_bigquery_objects
from tools.object_metadata_tool import get_object_metadata

In [ ]:
# main.py
from crewai import Crew
from agents  import schema_extractor_agent, object_metadata_agent, bq_mapper_agent
from tasks import AgentTasks
from langchain_core.utils.function_calling import convert_to_openai_tool
from tools.schema_listing_tool import list_bigquery_objects
from tools.object_metadata_tool import get_object_metadata
def main():
    project_id='nine-quality-test'
    # Step 1: Get all object names
    # Call the raw Python function directly with parameters
    object_list = list_bigquery_objects.func(project_id=project_id)
    results = []
    for dataset in object_list.keys():
        all_objects = [
            ("table", name) for name in object_list[dataset]["tables"]
        ] + [
            ("view", name) for name in object_list[dataset]["views"]
        ]  # You can later include procedures/functions too

   

        for object_type, object_name in all_objects:
            print(f"\n🔍 Analyzing {object_type.upper()} `{object_name}`...")

            # Step 2: Metadata extraction task
      
            metadata_task = AgentTasks.create_metadata_task(project_id, dataset, object_type, object_name, object_metadata_agent)
            metadata_crew = Crew(
                agents=[object_metadata_agent],
                tasks=[metadata_task],
                verbose=False
            )
            
            metadata = metadata_crew.kickoff()         

            # Step 3: Mapping to BigQuery
            mapping_task = AgentTasks.create_mapping_task(metadata, bq_mapper_agent)
            mapping_crew = Crew(
                agents=[bq_mapper_agent],
                tasks=[mapping_task],
                verbose=False
            )
            bq_result = mapping_crew.kickoff()

            results.append({
                "object": object_name,
                "type": object_type,
                "bigquery_schema": bq_result,
                "dataset":dataset
            })
            
            

    # Final report
    print("\n\n🧾 === BIGQUERY MIGRATION SUMMARY ===")
    for obj in results:
        print(f"\nObject: {obj['type'].upper()} `{obj['object']}`")
        print(obj["bigquery_schema"])

if __name__ == "__main__":
    main()


In [ ]:
!pip uninstall opentelemetry-sdk opentelemetry-api opentelemetry-exporter-otlp-proto -y
!pip install langchain-openai
!pip install langchain
!pip install langchain[google-genai]
!pip install langchain[vertexai]
!pip install -U langchain langchain-google-vertexai google-cloud-aiplatform
!pip install pyodbc
!pip install pymssql



!pip install \
  opentelemetry-api==1.24.0 \
  opentelemetry-sdk==1.24.0 \
  opentelemetry-exporter-otlp==1.24.0
